In [1]:
# Load AIEEE Data

In [1]:
%cd ../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


import sys  
sys.path.insert(0, 'Models/AIEEEGenderData/Preprocessing/')
from aieee_gender import *

In [3]:
aieee_df = AIEEEGenderData()

Load Models

In [4]:
#  other models
AIEEE_Muril = "Models/AIEEEGenderData/MurilBERT/SavedModel/MurilBERT"
CBSE_Muril = "Models/CBSEData/MurilBERT/SavedModel/MurilBERT"
ER_Muril = "Models/ERData/MurilBERT/SavedModel/BERTMuril"

In [5]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [6]:
from transformers import BertModel, AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [7]:
aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_Muril)

In [8]:
cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_Muril)

In [9]:
er_model = AutoModelForSequenceClassification.from_pretrained(ER_Muril)

In [10]:
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

Predict Gender

In [11]:
from transformers import pipeline
from sklearn.metrics import classification_report


In [12]:
import math

def print_results(rmodel, tokenizer, df):
    pipe = pipeline(task="text-classification",model=rmodel, tokenizer=tokenizer)
    n = df.shape[0]
    predictions = []
    for i in range(math.ceil(n/1000)):
        df1 = df[i*1000:(i+1)*(1000)]
        onway = pipe(list(df1['Name']), batch_size=256)
        predictions = predictions + onway
#     import pdb; pdb.set_trace()
    labels = []
    for pred in predictions:
        if pred['label'] == 'LABEL_0':
            labels.append(0)
        else:
            labels.append(1)
    
    true = list(df['Gender'])
    print(classification_report(true, labels, digits = 4))
    return 

In [86]:
print_results(aieee_model, tokenizer, aieee_df)

ValueError: Found input variables with inconsistent numbers of samples: [409286, 409000]

In [91]:
print_results(cbse_model, tokenizer, aieee_df)

              precision    recall  f1-score   support

         0.0     0.8646    0.9051    0.8844    175931
         1.0     0.9258    0.8932    0.9092    233355

    accuracy                         0.8983    409286
   macro avg     0.8952    0.8991    0.8968    409286
weighted avg     0.8995    0.8983    0.8985    409286



In [13]:
print_results(er_model, tokenizer, aieee_df)

              precision    recall  f1-score   support

         0.0     0.9468    0.9463    0.9466    175931
         1.0     0.9596    0.9599    0.9597    233355

    accuracy                         0.9541    409286
   macro avg     0.9532    0.9531    0.9531    409286
weighted avg     0.9541    0.9541    0.9541    409286

